# Part A. Build a baseline model (5 marks)

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

Submit your Jupyter Notebook with your code and comments.

In [2]:
import pandas as pd
import numpy as np
import statistics
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

The concrete data from lab 3 looked good, so just import and separate into variables and target

In [4]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
print(concrete_data.describe())

concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']].copy() # all columns except Strength
target = concrete_data['Strength'].copy() # Strength column

n_cols = predictors.shape[1]

            Cement  Blast Furnace Slag      Fly Ash        Water  \
count  1030.000000         1030.000000  1030.000000  1030.000000   
mean    281.167864           73.895825    54.188350   181.567282   
std     104.506364           86.279342    63.997004    21.354219   
min     102.000000            0.000000     0.000000   121.800000   
25%     192.375000            0.000000     0.000000   164.900000   
50%     272.900000           22.000000     0.000000   185.000000   
75%     350.000000          142.950000   118.300000   192.000000   
max     540.000000          359.400000   200.100000   247.000000   

       Superplasticizer  Coarse Aggregate  Fine Aggregate          Age  \
count       1030.000000       1030.000000     1030.000000  1030.000000   
mean           6.204660        972.918932      773.580485    45.662136   
std            5.973841         77.753954       80.175980    63.169912   
min            0.000000        801.000000      594.000000     1.000000   
25%            0.

Create and cycle through the model 50 times:

In [5]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

mse_dict = {}

for i in range(50):
    X = predictors.copy()
    y = target.copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    model = regression_model()
    model.fit(X_train, y_train, epochs = 50, verbose=0)
    mse_dict['%s' % (i)] = model.evaluate(X_test, y_test, verbose=0)

print(mse_dict)

{'0': 134.41053470747372, '1': 105.4270748891491, '2': 162.23566621947057, '3': 210.49791640988445, '4': 343.97233102545385, '5': 109.508859307249, '6': 253.1919022347163, '7': 135.74247963914593, '8': 118.05769851832714, '9': 1595.0617379494083, '10': 136.06504046492591, '11': 143.67157604624924, '12': 397.7677417742782, '13': 74.88003063510537, '14': 105.94560256513577, '15': 99.9568564899531, '16': 1579.200841613572, '17': 140.59496341939882, '18': 130.65422988940983, '19': 1271.9838855336013, '20': 357.0698796244501, '21': 164.32019420734886, '22': 154.92671028310042, '23': 1801.1124575716779, '24': 134.20801714554574, '25': 122.03412151645303, '26': 167.1350125309719, '27': 102.10038616587815, '28': 182.51458700729418, '29': 84.31373494959958, '30': 195.77181058717005, '31': 130.74554561874243, '32': 114.71026282943183, '33': 103.05721882940496, '34': 149.36752072429965, '35': 742.8632366094002, '36': 139.91971343697853, '37': 119.65222153154392, '38': 1957.2168000353965, '39': 12

Get the mean and standard deviation:

In [6]:
mse_mean = np.mean(list(mse_dict.values()))
mse_std = np.std(list(mse_dict.values()))
print('mean: ', mse_mean)
print('std: ', mse_std)

mean:  416.8549098111743
std:  702.9832543509636


# Part B. Normalize the data (5 marks)

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

In [7]:
#part B
# Normalize (standardize) the data
predictors_b = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']].copy() # all columns except Strength
target_b = concrete_data['Strength'].copy() # Strength column
predictors_norm = (predictors_b - predictors_b.mean()) / predictors_b.std()
print(predictors_norm.head())

mse_dict_b = {}

for i in range(50):
    X = predictors_norm.copy()
    y = target_b.copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    model = regression_model()
    model.fit(X_train, y_train, epochs = 50, verbose=0)
    mse_dict_b['%s' % (i)] = model.evaluate(X_test, y_test, verbose=0)

print(mse_dict_b)

     Cement  Blast Furnace Slag   Fly Ash     Water  Superplasticizer  \
0  2.476712           -0.856472 -0.846733 -0.916319         -0.620147   
1  2.476712           -0.856472 -0.846733 -0.916319         -0.620147   
2  0.491187            0.795140 -0.846733  2.174405         -1.038638   
3  0.491187            0.795140 -0.846733  2.174405         -1.038638   
4 -0.790075            0.678079 -0.846733  0.488555         -1.038638   

   Coarse Aggregate  Fine Aggregate       Age  
0          0.862735       -1.217079 -0.279597  
1          1.055651       -1.217079 -0.279597  
2         -0.526262       -2.239829  3.551340  
3         -0.526262       -2.239829  5.055221  
4          0.070492        0.647569  4.976069  
{'0': 289.4129419419372, '1': 403.5071170152584, '2': 369.16092185603765, '3': 513.1090123445085, '4': 455.8162665999823, '5': 550.6143425506295, '6': 612.5672307184213, '7': 276.36801157337175, '8': 476.8813140770378, '9': 578.6068125110614, '10': 272.4452457366252, '11':

In [8]:
mse_mean = np.mean(list(mse_dict_b.values()))
mse_std = np.std(list(mse_dict_b.values()))

print('mean: ', mse_mean)
print('std: ', mse_std)

mean:  373.8263201835164
std:  113.21384914960181


We can see that the mean between part A and B does not change significantly (374 versus 417), but that the standard deviation is much, much better (113 versus 703) after normalizing the data.

# Part C. Increate the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

In [9]:
# part c
mse_dict_c = {}

for i in range(50):
    X = predictors_norm.copy()
    y = target_b.copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    model = regression_model()
    model.fit(X_train, y_train, epochs = 100, verbose=0)
    mse_dict_c['%s' % (i)] = model.evaluate(X_test, y_test, verbose=0)

print(mse_dict_c)

{'0': 165.17648606778735, '1': 157.40032415791237, '2': 202.71686600867213, '3': 162.03134935496308, '4': 151.45225603912255, '5': 175.55761916274778, '6': 161.34232940180016, '7': 170.6853448071526, '8': 147.21175569700964, '9': 187.9368133544922, '10': 153.67470212275927, '11': 160.07854770302387, '12': 153.05425427878174, '13': 183.0321475032078, '14': 145.14479311847379, '15': 158.73056324091544, '16': 169.28535249086644, '17': 177.35748044109653, '18': 165.17771719034437, '19': 159.54949773399576, '20': 175.76124829536116, '21': 178.41526453703352, '22': 171.97906960792912, '23': 153.7045546349584, '24': 167.80124829733643, '25': 188.6328504247573, '26': 188.672940991843, '27': 150.39549230680495, '28': 187.51977484743188, '29': 205.00374773636605, '30': 164.9253614604666, '31': 169.2518089812936, '32': 197.43542070604838, '33': 160.66806222860095, '34': 176.21015979556975, '35': 165.77948644940523, '36': 186.98141573316457, '37': 154.29009298676425, '38': 149.57875673361966, '39'

In [10]:
mse_mean = np.mean(list(mse_dict_c.values()))
mse_std = np.std(list(mse_dict_c.values()))
print('mean: ', mse_mean)
print('std: ', mse_std)

mean:  173.55103826183327
std:  23.34728487468703


Compared to step B, we see that the mean is much lower (374 down to 174) and that the standard deviation is much better (from 113 to 23).

# Part D. Increase the number of hidden layers (5 marks)

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?

In [11]:
# define new regression model
def regression_model_3():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


mse_dict_d = {}

for i in range(50):
    X = predictors_norm.copy()
    y = target_b.copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    model = regression_model_3()
    model.fit(X_train, y_train, epochs = 50, verbose=0)
    mse_dict_d['%s' % (i)] = model.evaluate(X_test, y_test, verbose=0)

print(mse_dict_d)

{'0': 151.42876984926488, '1': 148.72983518927614, '2': 143.54292509702418, '3': 127.94386824820806, '4': 132.22278839407616, '5': 100.34690037205768, '6': 141.7069567831589, '7': 146.47032930935856, '8': 154.62109197227701, '9': 133.14708410960571, '10': 114.74259817870303, '11': 129.40106413511012, '12': 145.602660120498, '13': 115.6718837281261, '14': 149.4732472935155, '15': 132.73213166172064, '16': 123.63260776556811, '17': 124.81180421897123, '18': 114.0596081878761, '19': 108.47228465651231, '20': 129.66565704345703, '21': 125.25128771340577, '22': 137.35721864669455, '23': 107.86236492021185, '24': 121.43264123614166, '25': 145.08120078176356, '26': 147.63565201743907, '27': 116.7810576849385, '28': 93.72223367042912, '29': 129.95891551292445, '30': 135.45361098502445, '31': 132.5013286997971, '32': 131.64870398561547, '33': 96.24624426388046, '34': 137.50020409247637, '35': 127.91850394338466, '36': 126.72503839881675, '37': 130.47523755317366, '38': 145.4656885387828, '39': 

In [12]:
mse_mean = np.mean(list(mse_dict_d.values()))
mse_std = np.std(list(mse_dict_d.values()))
print('mean: ', mse_mean)
print('std: ', mse_std)

mean:  128.7363428768911
std:  14.705940711476353


Compared to step B, we see that the mean is much lower (374 down to 129) and that the standard deviation is much better (from 113 to 15).

### Conclusion

Overall, as we would expect, more layers and more training epochs result in better results (more consistent predictions). The change in the mean MSE between parts B and C, D is troubling, but the same data and same model was used for all predictions. Additionally, the consistency between parts C and D alleviates some of the concern.